### GPU check

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon May  3 22:50:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive 

### 해당 코드 실행 시 colab에서 실행중인 폴더의 /content/drive/My Drive가 구글 드라이브에 연결됨
base_dir = '/content/drive'
drive.mount(base_dir)

### SSH environment setting



In [ ]:
!pip install colab-ssh --upgrade
from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
launch_ssh_cloudflared(password="upstage")

In [ ]:
%cd /content/drive/MyDrive/boostcamp/pstage3/code

In [ ]:
# !pip install git+https://github.com/rwightman/pytorch-image-models.git
!pip install -U git+https://github.com/qubvel/segmentation_models.pytorch
!pip install wandb -qqq
!pip install -r requirements.txt

In [ ]:
import os
import random
import time
import json
import warnings 
import sys
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from utils import label_accuracy_score
import cv2
from importlib import import_module, reload

import argparse
import numpy as np
import pandas as pd

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp

from loss import create_criterion


# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import wandb


plt.rcParams['axes.grid'] = False

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())



pytorch version: 1.8.1+cu101
GPU 사용 가능 여부: True
Tesla V100-SXM2-16GB
1


## wandb login - 학습 모니터링 관리를 위함

In [ ]:
!wandb login

## CLI training & inference

### CLI training

In [ ]:
!python train.py --from_only_config True --config_path '/content/drive/MyDrive/boostcamp/pstage3/code/config.json'

### CLI inference and save

In [ ]:
!python inference.py --from_only_config True --config_path '/content/drive/MyDrive/boostcamp/pstage3/code/config.json'

## jupyer notebook training 
### Step by step

### Load configuration

In [ ]:
config_path = '/content/drive/MyDrive/Colab Notebooks/pstage3_segmentation/config.json'
with open(config_path, 'r') as f:
    config_dict = json.load(f)
args = argparse.Namespace()
d = vars(args)
for key, value in config_dict.items():
    d[key] = value
# add variable to argments for record
d['best_epoch'] = -1
d['best_mIoU'] = -1

print(args)

if args.is_wandb:
    if args.wandb_experiment_name == "":
        args.wandb_experiment_name = f'{args.model},enc:{args.encoder},loss:{args.criterion},optm:{args.optimizer},sche:{args.scheduler},bs:{args.batch_size},ep:{args.epochs}'
    wandb.init(project=args.wandb_project_name,
              group=args.wandb_group,
              name=args.wandb_experiment_name
              )

### Dataset, Augmentation, DataLoader

In [ ]:
import train
reload(import_module('train'))

train.seed_everything(args.random_seed)

# -- setting
use_cuda = torch.cuda.is_available()
device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

reload(import_module('dataset'))
# -- dataset
train_path = os.path.join(args.dataset_dir, 'train.json')
val_path = os.path.join(args.dataset_dir, 'val.json')
test_path =os.path.join(args.dataset_dir, 'test.json')

dataset_module = getattr(import_module("dataset"), args.dataset)  # default: BaseAugmentation
train_dataset = dataset_module(
    data_dir=args.dataset_dir,
    anotation_file=train_path,
    mode='train'
)
val_dataset = dataset_module(
    data_dir=args.dataset_dir,
    anotation_file=val_path,
    mode='val'
)
test_dataset = dataset_module(
    data_dir=args.dataset_dir,
    anotation_file=test_path,
    mode='test'
)
num_classes = train_dataset.num_classes  # 12
category_names = train_dataset.category_names

# -- augmentation
transform_module = getattr(import_module("dataset"), args.train_augmentation)
train_transform = transform_module()
train_dataset.set_transform(train_transform)

transform_module = getattr(import_module("dataset"), args.val_augmentation)
val_transform = transform_module()
val_dataset.set_transform(val_transform)

transform_module = getattr(import_module("dataset"), args.test_augmentation)
test_transform = transform_module()
test_dataset.set_transform(test_transform)

# --  DataLoader

# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=args.batch_size,
                                           shuffle=True,
                                           pin_memory=use_cuda,
                                           num_workers=4,
                                           drop_last=True,
                                           collate_fn=collate_fn)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                         batch_size=args.batch_size,
                                         shuffle=False,
                                         pin_memory=use_cuda,
                                         num_workers=4,
                                         drop_last=True,
                                         collate_fn=collate_fn)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=args.batch_size,
                                          pin_memory=use_cuda,
                                          num_workers=4,
                                          drop_last=True,
                                          collate_fn=collate_fn)

### model

In [ ]:
reload(import_module('model'))
model_module = getattr(import_module('model'), args.model)
model = model_module(num_classes=num_classes, args=args)
print(f'model: {args.model}')

# 구현된 model에 임의의 input을 넣어 output이 잘 나오는지 test
x = torch.randn([1, 3, 512, 512])
print("input shape : ", x.shape)
out = model(x).to(device)
print("output shape : ", out.size())

model = model.to(device)

model: PSPNet
input shape :  torch.Size([1, 3, 512, 512])
output shape :  torch.Size([1, 12, 512, 512])


### Loss and optimizer

In [ ]:
reload(import_module('loss'))
# Loss function 정의
criterion = create_criterion(args.criterion) # nn.CrossEntropyLoss()

# Optimizer 정의
opt_module = getattr(import_module("torch.optim"), args.optimizer)  # default: SGD
optimizer = opt_module(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=args.lr,
        weight_decay=1e-6
    )

### Training with validation

In [ ]:
reload(import_module('train'))
train.train(args, model, train_loader, val_loader, criterion, optimizer, device, category_names, val_dataset)
with open(args.saved_inference_config_path, 'w') as f_json:
    json.dump(vars(args), f_json)

Start training..
Epoch [1/5], Step [25/81], Loss: 1.1339
Epoch [1/5], Step [50/81], Loss: 0.9376


## Jupyter notebook inference
### 저장된 model 불러오기 (학습된 이후) 

In [ ]:
# get inference config from train_config
train_config_path = '/content/drive/MyDrive/Colab Notebooks/pstage3_segmentation/config.json'
with open(train_config_path, 'r') as f:
    config_dict = json.load(f)
# load inference config
inference_config_path = config_dict['saved_inference_config_path']
with open(inference_config_path, 'r') as f:
    inference_config_dict = json.load(f) 

args = argparse.Namespace()
d = vars(args)
for key, value in inference_config_dict.items():
    d[key] = value
# add variable to argments for record
# d['best_epoch'] = -1
# d['best_mIoU'] = -1
print(args)

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

dataset_module = getattr(import_module("dataset"), args.dataset)

# -- dataset

test_path =os.path.join(args.dataset_dir, 'test.json')
test_dataset = dataset_module(
    data_dir=args.dataset_dir,
    anotation_file=test_path,
    mode='test'
)
num_classes = test_dataset.num_classes  # 12
category_names = test_dataset.category_names

# -- augmentation

transform_module = getattr(import_module("dataset"), args.test_augmentation)
test_transform = transform_module()
test_dataset.set_transform(test_transform)

# -- data Loader
# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))    

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                      batch_size=args.batch_size,
                                      pin_memory=use_cuda,
                                      num_workers=4,
                                      drop_last=True,
                                      collate_fn=collate_fn)

In [ ]:
import inference
reload(import_module('inference'))

# load model
model = inference.load_model(device, num_classes, args).to(device)

# 추론을 실행하기 전에는 반드시 설정 (batch normalization, dropout 를 평가 모드로 설정)
# model.eval()

In [ ]:
# 첫번째 batch의 추론 결과 확인
for imgs, image_infos in test_loader:
    image_infos = image_infos
    temp_images = imgs
    
    model.eval()
    # inference
    outs = model(torch.stack(temp_images).to(device))
    oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
    
    break

i = 1
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 16))

print('Shape of Original Image :', list(temp_images[i].shape))
print('Shape of Predicted : ', list(oms[i].shape))
print('Unique values, category of transformed mask : \n', [{int(i),category_names[int(i)]} for i in list(np.unique(oms[i]))])

# Original image
ax1.imshow(temp_images[i].permute([1,2,0]))
ax1.grid(False)
ax1.set_title("Original image : {}".format(image_infos[i]['file_name']), fontsize = 15)

# Predicted
ax2.imshow(oms[i])
ax2.grid(False)
ax2.set_title("Predicted : {}".format(image_infos[i]['file_name']), fontsize = 15)

plt.show()

### Inference

In [ ]:
file_names, preds = inference.inference(model, test_loader, device)

### submission

In [ ]:
reload(import_module('inference'))

if not os.path.isdir(args.submission_dir):
    os.mkdir(args.submission_dir)

submission_file_path = os.path.join(args.submission_dir, 'sample_submission.csv')
# sample_submisson.csv 열기
submission = pd.read_csv(submission_file_path, index_col=None)

# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

submission, output_file_path = inference.save_submission_csv(submission, args)

# auto submission
desc = f'''from colab
        model:{args.model}, encoder:{args.encoder}, loss:{args.criterion}, optimizer:{args.optimizer}, lr:{args.lr}, epoch:{args.best_epoch}/{args.epochs}, best mIoU:{args.best_mIoU}, batch size:{args.batch_size}\n
        train augmentation:{args.train_augmentation}, val augmentation:{args.val_augmentation}, test augmentation:{args.test_augmentation} '''
user_key = args.submission_user_key

# -- submit to server
inference.submit(user_key, output_file_path, desc)

## Reference

